<p style="font-size:32px; font-weight: bolder; text-align: center"> Colored-noise methods </p>

This notebook provides a hands-on counterpart to the "Colored-noise methods" lecture for the MOOC "Path Integrals in Atomistic Modeling". If you haven't done so already, check the [getting started](0-getting_started.ipynb) notebook to make sure that the software infrastructure is up and running. 

The different sections in this notebook match the parts this lecture is divided into:

1. [Generalized Langevin Equations](#gle)
2. [Equilibrium GLE sanpling](#equilibrium)
3. [Non-Equilibrium GLE sanpling](#non-equilibrium)
4. [Combining GLE and PIMD](#pi-gle)
5. [Dynamical properties](#dynamics)

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import ase, ase.io
import chemiscope
import pimdmooc
pimdmooc.add_ipi_paths()

<a id="gle"> </a>

# Generalized Langevin Equations 

This section provides a brief demonstration of the possibility of computing properties of a generalized Langevin equation written in the extended-phase-space form, as a function of the drift matrix $\mathbf{A}_p$ and the diffusion matrix $\mathbf{B}_p$ (the latter being usually fixed by a fluctuation-dissipation relation.

A matrix $\mathbf{M}_p$ coupling physical momentum $p$ and extended momenta $\mathbf{s}$ can be written down as a combination of blocks, following the convention

$$
\begin{array}{c|cc}
      &    p   &   \mathbf{s} \\ \hline
   p  & m_{pp} &  \mathbf{m}_p^T \\
\mathbf{s} & \bar{\mathbf{m}}_p &  \mathbf{M}\\
\end{array}
$$

In [ ]:
# this splits up a Mp matrix into the four blocks describing interactions between p and s
def gle_split(Mp):
    """ Splits a matrix in the various blocks """
    return Mp[:1,:1], Mp[:1, 1:], Mp[1:, :1], Mp[1:,1:]

The simplest property that can be computed from the GLE matrices are the memory kernels of the associated non-Markovian formulation. The friction kernel reads

$$
K(t) = a_{pp} \delta(t) - \mathbf{a}_p^T e^{-t \mathbf{A}} \bar{\mathbf{a}}_p
$$

and its Fourier transform 

$$
K(\omega) = 2 a_{pp} -2 \mathbf{a}_p^T \frac{\mathbf{A}}{\mathbf{A}^2+\omega^2} \bar{\mathbf{a}}_p
$$

In [ ]:
def Kt(Ap, t, with_delta=False):
    app, ap, bap, A = gle_split(Ap)
    return (app[0,0] if with_delta and t==0 else 0) - (ap@sp.linalg.expm(-A*t)@bap)[0,0]

def Kw(Ap, w):
    app, ap, bap, A = gle_split(Ap)
    return (2*app  - 2*(ap@A)@np.linalg.solve(A@A+np.eye(len(A))*w**2,bap))[0,0]

## Memory kernels

Different analytical forms of memory kernel can be obtained with appropriate parameterizations of the drift matrix. The ones given below yield $K(t)$ that are exponential, $K(t)=\gamma e^{-t/\tau}$ or different types of Dirac-$\delta$-like functions, that give a peaked memory kernel in the frequency domain (the corresponding functional form is rather cumbersome, you can find a thorough discussion [here](https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/152344/eth-2145-02.pdf)) - the functional form has been slightly modified to give more intuitive link between the parameters and the shape of $K(\omega)$. In all these cases, the parameter `app`, corresponding to $a_{pp}$, introduces a non-Markovian term in the kernel.

In [ ]:
def Ap_exp(gamma, tau, app=0):
    """ Drift matrix for an exponential memory kernel.
    gamma: intensity of the friction
    tau: time scale of the exponential decay
    app: Markovian term
    """
    return np.asarray( [ [app,-np.sqrt(gamma/tau) ], [np.sqrt(gamma/tau), 1/tau ]])
def Ap_delta(gamma, omega0, domega, app=0):
    """ Drift matrix for a delta-like memory kernel.  
    gamma: intensity of the friction
    omega0: frequency-domain center of the K(w) peak
    domega: width of the peak
    app: Markovian term
    """
    return np.asarray( [ [app, np.sqrt(gamma*domega/2), np.sqrt(gamma*domega/2) ], 
                         [-np.sqrt(gamma*domega/2),  domega, omega0 ],
                         [-np.sqrt(gamma*domega/2), -omega0, domega ]
                       ])
def Ap_delta_alt(gamma, omega0, domega, app=0):
    """ Drift matrix for a delta-like memory kernel. Alternative form with K(0)=0. 
    gamma: intensity of the friction
    omega0: frequency-domain center of the K(w) peak
    domega: width of the peak
    app: Markovian term
    """
    return np.asarray( [ [app,np.sqrt(gamma*domega/2), 0 ], 
                         [-np.sqrt(gamma*domega/2),  domega, omega0 ],
                         [0, -omega0, 0 ]
                       ])

Below you can plot $K(\omega)$ for the three functional forms above. Play around with the parameters to verify their effect on the shape of the memory kernel spectrum. 

In [ ]:
wgrid = np.geomspace(1e-3,1e3,200)
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
# defaults: Ap_delta_alt(1, 1, 1, 1e-8)
ax.loglog(wgrid, [Kw(Ap_delta_alt(1, 1, 0.1, 1e-8), w) for w in wgrid], 'r-' )  
# defaults: Ap_delta(1, 0.1, 0.01, 1e-8)
ax.loglog(wgrid, [Kw(Ap_delta(1, 0.1, 0.01, 1e-8), w) for w in wgrid], 'b-' ) 
# defaults: Ap_exp(1, 1, 0)
ax.loglog(wgrid, [Kw(Ap_exp(1, 1, 0), w) for w in wgrid], 'k-' )
ax.set_xlabel(r"$\omega$ / a.u."); ax.set_ylabel(r"$K(\omega)$ / a.u."); 
fig.savefig("/tmp/gle-exp-delta.pdf")

An important idea that makes it easy to reuse GLE parameters for different systems is that it is possible to "translate" the shape of $K(\omega)$ by scaling it by a factor $\alpha$. This is essentially a change of units, so scaling the kernel moves the curve right and up in the $(\omega, K)$ plane

In [ ]:
wgrid = np.geomspace(1e-3,1e3,200)
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.loglog(wgrid, [Kw(Ap_delta_alt(1,1,1,1e-4), w) for w in wgrid], 'r-' )
ax.loglog(wgrid, [Kw(0.1*Ap_delta_alt(1,1,1,1e-4), w) for w in wgrid], 'r:' )
ax.loglog(wgrid, [Kw(10*Ap_delta_alt(1,1,1,1e-4), w) for w in wgrid], 'r--' )
ax.set_xlabel(r"$\omega$ / a.u."); ax.set_ylabel(r"$K(\omega)$ / a.u."); 
fig.savefig("/tmp/gle-alpha-shift.pdf")

<p style="color:blue; font-weight:bold">
In the case of the analytical memory functions above, you can actually also mimic the effect of scaling by changing the value of the parameters. What parameters should you use for `Ap_delta` below, so that the blue curve becomes identical to the red curve?
</p>

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.loglog(wgrid, [Kw(0.1*Ap_delta(1,1,0.1,1e-4), w) for w in wgrid], 'r--' )

# modify the parameters below ↓ ↓ ↓ ↓, corresponding to gamma, omega0, domega, app
ax.loglog(wgrid, [Kw(Ap_delta(1,1,1,0), w) for w in wgrid], 'b:' )

## GLE integration

In [ ]:
class VVIntegrator(object):
    """ Velocity-Verlet integrator """
    def __init__(self, force, dt, q):
        
        self.force = force
        self.dt = dt
        self.f = force(q)
    
    def step(self, q, p):
        
        p[:] += self.f * self.dt *0.5
        q[:] += p * self.dt
        self.f = self.force(q)
        p[:] += self.f * self.dt*0.5            
        
class GLEIntegrator(object):
    """ Finite time-step GLE integrator for a free particle """
    def __init__(self, Ap, Cp, dt):
        self.ns = len(Ap)-1
        self.Ap = Ap
        self.Cp = Cp
        self.dt = dt
        self.T = sp.linalg.expm(-Ap*self.dt*0.5)
        self.S = sp.linalg.cholesky(self.Cp - self.T @ self.Cp @self.T.T).T
    
    def step(self, p, s):        
        ps = np.vstack([p,s])
        # stores the "GLE force" contribution for analysis
        self._rf = self.T @ ps - ps + self.S @ np.random.normal(size=(self.ns+1, len(p)))
        ps += self._rf        
        p[:] = ps[0]
        s[:] = ps[1:]

In [ ]:
q = np.asarray([0.,0.,0.])
p = np.asarray([0.,0.,0.])
s = np.asarray([[0.,0.,0.]])

In [ ]:
lq = []
lp = []
Ap = Ap_exp(0.2, 0.5, 0.2)
GLE = GLEIntegrator(Ap, np.eye(2), 0.1)
VV = VVIntegrator(lambda x:0.*x, 0.1, q)

In [ ]:
nstep = 40000
lq = np.zeros((nstep, len(q)))
lp = np.zeros((nstep, len(p)))
lf = np.zeros((nstep, len(p)))
for istep in range(nstep):
    lq[istep] = q; lp[istep] = p
    GLE.step(p,s)
    lf[istep] = GLE._rf[0]/(0.5*GLE.dt)
    VV.step(q,p)
    GLE.step(p,s)    
lq = np.asarray(lq)
lp = np.asarray(lp)

In [ ]:
plt.plot(lp[:,0])

In [ ]:
np.std(lp[:,0])

In [ ]:
plt.plot(lf[-200:,0])

In [ ]:
cvv = autocorrelate(lp[(nstep//10):,0], 0)

In [ ]:
vv = correlate(lp[(nstep//10):,0], lp[(nstep//10):,0], 0, 0)
vf = -(correlate(lf[(nstep//10):,0], lp[(nstep//10):,0], 0, 0) - correlate(lp[(nstep//10):,0],lf[(nstep//10):,0], 0, 0))/2

In [ ]:
Kt(Ap, 0)

In [ ]:
kk = np.asarray([Kt(Ap, VV.dt*t) for t in range(len(vv))])

In [ ]:
ee = np.asarray( [ GLE.dt*(vv[:i]*kk[:i][::-1]).sum() for i in range(len(kk))] )

In [ ]:
#plt.plot(vf[:100], 'b')
plt.plot(vv[:100], 'r')
plt.plot(kk[:100]*1e4, 'k')
plt.plot(vf[:100], 'b.')
plt.plot((vf - Ap[0,0]*vv)[:100], 'g.')
plt.plot(ee[:100], 'k--')